In [1]:
import org.apache.spark._
import org.apache.spark.rdd.RDD
import org.apache.spark.util.IntParam
import org.apache.spark.graphx._
import org.apache.spark.graphx.util.GraphGenerators

In [2]:
//Lets define movie class//
case class Movie(
    tconst:String, 
    titleType:String, 
    primaryTitle:String, 
    originalTitle:String, 
    isAdult:String, 
    startYear:String, 
    endYear:String, 
    runtimeMinutes:String, 
    genre:String
)

defined class Movie


In [3]:
//Lets define rating class//
case class Rating(
    tconst:String, 
    averageRating:Float, 
    noOfVotes:Long
)

defined class Rating


In [4]:
//Lets define actors class//
case class Actors(
    nconst:String, 
    primaryName:String, 
    birthYear:String, 
    deathYear:String,
    primaryProfession:String,
    knownForTitles:List[String]
)

defined class Actors


In [5]:
//Lets define actors class//
case class crew(
    tconst:String, 
    directors:String, 
    writers:String
)

defined class crew


In [6]:
//Some parsing is done to add each line to respective objects
def parseMovie(str: String): Movie = {
 val line = str.split("\t")
 Movie(line(0).substring(2), line(1), line(2), line(3), line(4), line(5), line(6), line(7), line(8))
}

parseMovie: (str: String)Movie


In [7]:
def parseRating(str: String): Rating = {
 val line = str.split("\t")
 Rating(line(0).substring(2), line(1).toFloat, line(2).toLong)
}

parseRating: (str: String)Rating


In [8]:
def parseActors(str: String): Actors = {
 val line = str.split("\t")
 Actors(line(0).substring(2), line(1), line(2), line(3), line(4), line(5).split(",").toList)
}

parseActors: (str: String)Actors


In [9]:
def parseCrew(str: String): crew = {
 val line = str.split("\t")
 crew(line(0).substring(2), line(1), line(2))
}

parseCrew: (str: String)crew


In [10]:
//load the datasets - movie
val movies = sc.textFile("/home/prakass1/imdb_dataset/data_title.tsv")
val top_row = movies.first()
val movies1 = movies.filter(x => x!= top_row)

Waiting for a Spark session to start...

movies = /home/prakass1/imdb_dataset/data_title.tsv MapPartitionsRDD[1] at textFile at <console>:37
top_row = tconst	titleType	primaryTitle	originalTitle	isAdult	startYear	endYear	runtimeMinutes	genres
movies1 = MapPartitionsRDD[2] at filter at <console>:39


MapPartitionsRDD[2] at filter at <console>:39

In [11]:
//load the datasets - rating
val rating = sc.textFile("/home/prakass1/imdb_dataset/data_rating.tsv")
val rating_row = rating.first()
val rating1 = rating.filter(y => y!= rating_row)

Waiting for a Spark session to start...

rating = /home/prakass1/imdb_dataset/data_rating.tsv MapPartitionsRDD[4] at textFile at <console>:37
rating_row = tconst	averageRating	numVotes
rating1 = MapPartitionsRDD[5] at filter at <console>:39


MapPartitionsRDD[5] at filter at <console>:39

In [27]:
//load the datasets - actors
//val actors = sc.textFile("/home/prakass1/imdb_dataset/data_names.tsv")
//val actor_row = actors.first()
//val actors1 = actors.filter(x => x!= actor_row)

actors = /home/prakass1/imdb_dataset/data_names.tsv MapPartitionsRDD[31] at textFile at <console>:40
actor_row = nconst	primaryName	birthYear	deathYear	primaryProfession	knownForTitles
actors1 = MapPartitionsRDD[32] at filter at <console>:42


MapPartitionsRDD[32] at filter at <console>:42

In [12]:
//load the datasets - actors
val crew = sc.textFile("/home/prakass1/imdb_dataset/data_crew.tsv")
val crew_row = crew.first()
val crew1 = crew.filter(x => x!= crew_row)

crew = /home/prakass1/imdb_dataset/data_crew.tsv MapPartitionsRDD[7] at textFile at <console>:39
crew_row = tconst	directors	writers
crew1 = MapPartitionsRDD[8] at filter at <console>:41


MapPartitionsRDD[8] at filter at <console>:41

In [13]:
val moviesRDD = movies1.map(parseMovie).cache()

moviesRDD = MapPartitionsRDD[9] at map at <console>:44


MapPartitionsRDD[9] at map at <console>:44

In [14]:
val ratingRDD = rating1.map(parseRating).cache()

ratingRDD = MapPartitionsRDD[10] at map at <console>:44


MapPartitionsRDD[10] at map at <console>:44

In [15]:
val crewRDD = crew1.map(parseCrew).cache()

crewRDD = MapPartitionsRDD[11] at map at <console>:43


MapPartitionsRDD[11] at map at <console>:43

In [32]:
//val actorsRDD = actors1.map(parseActors).cache()

actorsRDD = MapPartitionsRDD[39] at map at <console>:44


MapPartitionsRDD[39] at map at <console>:44

In [16]:
val titles = moviesRDD.map(movies => (movies.tconst.toLong,movies.primaryTitle)).distinct()

titles = MapPartitionsRDD[15] at distinct at <console>:46


MapPartitionsRDD[15] at distinct at <console>:46

In [1]:
titles.take(2)

[(766914,Erkki vastaan Paavo), (720406,Clifton's Sugar Mama)]

In [17]:
val movieRating = ratingRDD.map(rating => (rating.tconst,rating.averageRating,rating.noOfVotes)).distinct()

movieRating = MapPartitionsRDD[19] at distinct at <console>:46


MapPartitionsRDD[19] at distinct at <console>:46

In [3]:
movieRating.take(1)

[(3250554,5.8,22)]

In [2]:
//val movieActors = actorsRDD.map(actors => (actors.nconst,actors.primaryName,actors.knownForTitles)).distinct()

movieActors = MapPartitionsRDD[31] at distinct at <console>:46


MapPartitionsRDD[31] at distinct at <console>:46

In [23]:
//movieActors.take(1)

Name: Syntax Error.
Message: 
StackTrace: 

In [18]:
val movieCrew = crewRDD.map(crew => (crew.tconst,crew.directors)).distinct()

movieCrew = MapPartitionsRDD[23] at distinct at <console>:45


MapPartitionsRDD[23] at distinct at <console>:45

In [40]:
movieCrew.take(2)

[(2128519,\N), (4801526,\N)]

In [19]:
//Make a votes based vector where vote holds the weight
val votes = ratingRDD.map(rating => ((rating.tconst, rating.averageRating), rating.noOfVotes)).distinct

votes = MapPartitionsRDD[27] at distinct at <console>:47


MapPartitionsRDD[27] at distinct at <console>:47

In [27]:
votes.take(1)

[((0323097,8.4),13)]

In [42]:
val movieCrewEdge = movieCrew.map {
    case ((tconst,directors)) => Edge(tconst.toLong,directors.toLong)
}

movieCrewEdge = MapPartitionsRDD[61] at map at <console>:47


MapPartitionsRDD[61] at map at <console>:47

In [20]:
val edges = votes.map {
 case ((tconst, averageRating), noOfVotes) =>Edge(tconst.toLong,averageRating.toLong,noOfVotes.toLong) }

edges = MapPartitionsRDD[28] at map at <console>:48


MapPartitionsRDD[28] at map at <console>:48

In [29]:
edges.take(1)

[Edge(323097,8,13)]

In [21]:
val graph = Graph(titles, edges)  

graph.vertices.take(2)

graph = org.apache.spark.graphx.impl.GraphImpl@6da9afdb


[(1115179,Introduction), (354796,Parade de quarte)]

In [ ]:
graph.edges.take(10)

## Number of movies in the dataset

In [9]:
val noOfMovies = graph.numVertices

noOfMovies = 5040244


5040244

## Number of votes

In [10]:
val values = graph.numEdges

values = 838182


838182

### Which movieId votes are greater than 15Lakh?

In [ ]:
val votes = 1500000

### Get the Highest Voted Movie

In [7]:
val highVotedMovies = graph.edges.filter { case ( Edge(tconst, averageRating,noOfVotes))=> noOfVotes >= 1500000}.take(10)

highVotedMovies = Array(Edge(111161,9,1957480), Edge(468569,9,1929695), Edge(110912,8,1530948), Edge(137523,8,1569401), Edge(1375666,8,1714048))


[Edge(111161,9,1957480), Edge(468569,9,1929695), Edge(110912,8,1530948), Edge(137523,8,1569401), Edge(1375666,8,1714048)]

#### Get the movie name from edge

In [12]:
graph.vertices.filter{ case (tconst, primaryTitle) => tconst == 111161 }.take(1)

[(111161,The Shawshank Redemption)]

In [ ]:
val ranks = graph.pageRank(0.0001).vertices
ranks
  .join(titles)
  .sortBy(_._2._1, ascending=false) // sort by the rank
  .take(1)